# Multinational Retail Data Centralisation
## Task Document to Run Executions of Project Work.

#### Disclaimer: Due to the project work being done at an earlier stage before Task 7 of Milestone 2. I will hope to back log my steps at the end of the project (if there is time). Since I am on the fast-track programme, time isn't a luxury I have.

In [2]:
import  pandas              as pd
import  numpy               as np

from    data_utils         import DataConnector
from    data_extraction    import DataExtractor
from    data_cleaning      import DataCleaning

## TASK 7
### Extracting Orders Table

In [3]:
dc              = DataConnector()
de              = DataExtractor()
dcl             = DataCleaning()

name            = "db_creds.yaml" 
credentials     = dc.read_db_creds(name)
engine          = dc.init_db_engine(credentials)
table_names     = dc.list_db_tables(engine)
column          = 2

dc.list_db_tables(engine)
de.read_rds_table(table_names,column, engine)

,level_0,index,date_uuid,first_name,last_name,user_uuid,card_number,store_code,product_code,1,product_quantity
0,0,0,9476f17e-5d6a-4117-874d-9cdb38ca1fa6,None,None,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8,30060773296197,BL-8387506C,R7-3126933h,NaN,3
1,1,1,0423a395-a04d-4e4a-bd0f-d237cbd5a295,None,None,8fe96c3a-d62d-4eb5-b313-cf12d9126a49,349624180933183,WEB-1388012W,C2-7287916l,NaN,2
2,2,2,65187294-bb16-4519-adc0-787bbe423970,None,None,fc461df4-b919-48b2-909e-55c95a03fe6b,3529023891650490,CH-01D85C8D,S7-1175877v,NaN,2
3,3,3,579e21f7-13cb-436b-83ad-33687a4eb337,None,None,6104719f-ef14-4b09-bf04-fb0c4620acb0,213142929492281,CL-C183BE4B,D8-8421505n,NaN,2
4,4,4,00ab86c3-2039-4674-b9c1-adbcbbf525bd,None,None,9523a6d3-b2dd-4670-a51a-36aebc89f579,502067329974,SO-B5B9CB3B,B6-2596063a,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...
120118,110549,110548,f0e8fff6-9998-4661-954b-0e258e09d33c,None,None,95c74b0a-d495-4359-b1c0-e2da511e8403,575421945446,KA-FA7ED3B8,C9-6827622o,NaN,4
120119,82164,82164,1c80940a-d186-4ba9-9daa-8abd1aceae32,None,None,5d6fa6fe-e583-4baf-8bbb-d1dd6e2b551f,4971858637664481,WA-A41DA979,I0-1146408B,NaN,1
120120,97599,97599,58598aca-049c-418e-8e39-46327634a7f1,Sharon,Miller,48b7f1fc-db13-4611-ad8e-3dac0b759488,4971858637664481,WEB-1388012W,A4-5443400b,NaN,4
120121,106591,106591,3a76f661-0707-4fbc-9862-f21d3249f581,None,None,51c0b538-7ded-4697-8e84-9f7aa13f9112,4971858637664481,SO-6D328417,E9-2782979e,NaN,4


### **CLEANING TIME**
Before we can upload this to our SALES DATA Database. We need to clean up the code.

***Let's get those gloves on.***

In [7]:
database = de.read_rds_table(table_names,column, engine)
database = database.drop(columns= "level_0")

#### Now that we have the orders table as a PD Dataframe. We can upload it to our SALES DATA Database.

In [8]:

#connect to local server
sql_name        = "orders_table"
local_name      = "db_creds_local.yaml" 
credentials     = dc.read_db_creds(local_name)
engine          = dc.init_db_engine(credentials)

#UPLOAD
dc.upload_to_db(database, sql_name, engine)

***
## TASK 8
***
### Extracting Date of events (activities) of any purchase/orders
###### We have created a function using the api method to manage this connection. **(we should consider updating the old API method to a more streamlined version where we can control the URL more fluidly in this notebook.)**

#### In order to create this table, we are extracting from the amazon S3 server as URL.
#### Thus, we will use the reccomended method described in the Boto3 1.33.11 Documentation found at, https://boto3.amazonaws.com/v1/documentation/api/latest/guide/s3-presigned-urls.html

In [7]:
import  pandas              as pd
import requests    # To install: pip install requests

LINK = "https://data-handling-public.s3.eu-west-1.amazonaws.com/date_details.json"
def extract_from_s3_LINK():
    if LINK is not None:
        response = requests.get(LINK)
        data = pd.DataFrame(response.json())
    return data

extract_from_s3_LINK()

,timestamp,month,year,day,time_period,date_uuid
0,22:00:06,9,2012,19,Evening,3b7ca996-37f9-433f-b6d0-ce8391b615ad
1,22:44:06,2,1997,10,Evening,adc86836-6c35-49ca-bb0d-65b6507a00fa
2,10:05:37,4,1994,15,Morning,5ff791bf-d8e0-4f86-8ceb-c7b60bef9b31
3,17:29:27,11,2001,6,Midday,1b01fcef-5ab9-404c-b0d4-1e75a0bd19d8
4,22:40:33,12,2015,31,Evening,dfa907c1-f6c5-40f0-aa0d-40ed77ac5a44
...,...,...,...,...,...,...
120156,22:56:56,11,2022,12,Evening,d6c4fb31-720d-4e94-aa6b-dcbcb85f2bb7
120157,18:25:20,5,1997,31,Evening,f7722027-1aae-49c3-8f8d-853e93f9f3e6
120158,18:21:40,9,2011,13,Evening,4a3b9851-52e1-463c-ac81-1960f141444e
120159,19:10:53,7,2013,12,Evening,64974909-0d4b-42a2-822a-73b5695e8bfb


#### Now that we know we can correctly extract the data. We can commence **CLEANING**.

#### Now that we have cleaned the **DATA** we can transport these functions into our "data_extraction.py" and "data_cleaning.py" documents. After which we can commence the uploading. <br> As always, we will contain all imports and neccessary variables in this cell, as a self contained operation.

In [1]:
from    data_utils          import  DataConnector
from    data_extraction     import  DataExtractor
from    data_cleaning       import  DataCleaning

def     upload_to_dim_dates():
                #import class variables
                dc              = DataConnector()
                de              = DataExtractor()
                dcl             = DataCleaning()

                #extracttion
                LINK            = "https://data-handling-public.s3.eu-west-1.amazonaws.com/date_details.json"
                database        = de.extract_from_s3_LINK(LINK)

                #cleaning


                #local server
                sql_name        = "event_dates"
                local_name      = "db_creds_local.yaml" 
                credentials     = dc.read_db_creds(local_name)
                engine          = dc.init_db_engine(credentials)

                #UPLOAD
                dc.upload_to_db(database, sql_name, engine)

upload_to_dim_dates()